In [96]:
import cv2
import numpy as np

In [97]:
def get_line_pixels_vertical(image, x0, y0, x1, y1):
    # Implement the Bresenham's line algorithm
    points = []
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    x, y = x0, y0
    sx = -1 if x0 > x1 else 1
    sy = -1 if y0 > y1 else 1
    if dx > dy:
        err = dx / 2.0
        while x != x1:
            if 0 <= x < len(image[0]) and 0 <= y < len(image):
                if 0 <= x-1 < len(image[0]) and 0 <= x+1 < len(image[0]):
                    points.append((x, y))
            err -= dy
            if err < 0:
                y += sy
                err += dx
            x += sx
    else:
        err = dy / 2.0
        while y != y1:
            if 0 <= x < len(image[0]) and 0 <= y < len(image):
                if 0 <= x-1 < len(image[0]) and 0 <= x+1 < len(image[0]):
                    points.append((x, y))
            err -= dx
            if err < 0:
                x += sx
                err += dy
            y += sy
    if 0 <= x < len(image[0]) and 0 <= y < len(image):
        if 0 <= x-1 < len(image[0]) and 0 <= x+1 < len(image[0]):
            points.append((x, y))

    # Get the pixel values
    pixels = [(image[y][x-1], image[y][x+1]) for x, y in points]

    return pixels

In [98]:
def get_line_pixels_horizontal(image, x0, y0, x1, y1):
    # Implement the Bresenham's line algorithm
    points = []
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    x, y = x0, y0
    sx = -1 if x0 > x1 else 1
    sy = -1 if y0 > y1 else 1
    if dx > dy:
        err = dx / 2.0
        while x != x1:
            if 0 <= x < len(image[0]) and 0 <= y < len(image):
                if 0 <= y-1 < len(image) and 0 <= y+1 < len(image):
                    points.append((x, y))
            err -= dy
            if err < 0:
                y += sy
                err += dx
            x += sx
    else:
        err = dy / 2.0
        while y != y1:
            if 0 <= x < len(image[0]) and 0 <= y < len(image):
                if 0 <= y-1 < len(image) and 0 <= y+1 < len(image):
                    points.append((x, y))
            err -= dx
            if err < 0:
                x += sx
                err += dy
            y += sy
    if 0 <= x < len(image[0]) and 0 <= y < len(image):
        if 0 <= y-1 < len(image) and 0 <= y+1 < len(image):
            points.append((x, y))

    # Get the pixel values
    pixels = [(image[y-1][x], image[y+1][x]) for x, y in points]

    return pixels

In [99]:
# Load the image
image = cv2.imread('cropped/AA AA-302-[0].jpg', 0)  # 0 loads the image in grayscale

In [100]:
all_verticals = []
for angle in range(80, 100):
    all_contrasts = []

    # print("Angle: ", angle)
    # Convert the angle to radians
    angle = np.radians(angle)

    for i in range(image.shape[1]):
        # Define the line parameters (angle and starting point)
        x0, y0 = i, 0  # Replace with your desired starting point

        # Calculate the end point of the line
        x1 = int(x0 + 200 * np.cos(angle))
        y1 = int(y0 + 200 * np.sin(angle))

        pixels = get_line_pixels_vertical(image, x0, y0, x1, y1)
        pixels = np.array(pixels, dtype=np.int16)

        if len(pixels) == 0:
            continue

        total_contrast = pixels[:, 0] - pixels[:, 1]
        all_contrasts.append(total_contrast.mean())

    all_contrasts = np.array(all_contrasts)
    # print(all_contrasts.argmin(), all_contrasts.min())
    # print(all_contrasts.argmax(), all_contrasts.max())
    length = all_contrasts.shape[0]
    argmin = all_contrasts[:40].argmin()
    min = all_contrasts[:40].min()
    argmax = all_contrasts[length-40:].argmax() + length-40
    max = all_contrasts[length-40:].max()
    all_verticals.append((argmin, min, argmax, max, max-min))

In [101]:
all_horizontals = []
for angle in range(-15, 15):
    all_contrasts = []

    # print("Angle: ", angle)
    # Convert the angle to radians
    angle = np.radians(angle)

    for i in range(-20, image.shape[0]+20):
        # Define the line parameters (angle and starting point)
        x0, y0 = 0, i  # Replace with your desired starting point

        # Calculate the end point of the line
        x1 = int(x0 + 500 * np.cos(angle))
        y1 = int(y0 + 500 * np.sin(angle))

        pixels = get_line_pixels_horizontal(image, x0, y0, x1, y1)
        pixels = np.array(pixels, dtype=np.int16)

        if len(pixels) < image.shape[1] * 0.7:
            continue

        total_contrast = pixels[:, 0] - pixels[:, 1]
        all_contrasts.append(total_contrast.mean())

    all_contrasts = np.array(all_contrasts)
    # print(all_contrasts.argmin(), all_contrasts.min())
    # print(all_contrasts.argmax(), all_contrasts.max())
    length = all_contrasts.shape[0]
    argmin = all_contrasts.argmin()
    min = all_contrasts.min()
    argmax = all_contrasts.argmax()
    max = all_contrasts.max()
    all_horizontals.append((argmin, min, argmax, max, max-min))

In [102]:
all_verticals = np.array(all_verticals)
best_lines = all_verticals[all_verticals[:, 4].argmax()]

# Define the line parameters (angle and starting point)
angle = 80 + all_verticals[:, 4].argmax()
x0_left, y0_left = int(best_lines[0]), 0
x0_right, y0_right = int(best_lines[2]), 0

print("Angle: ", angle)
print(best_lines)

# Convert the angle to radians
angle = np.radians(angle)

# Calculate the end point of the lines
x1_left = int(x0_left + 200 * np.cos(angle))
y1_left = int(y0_left + 200 * np.sin(angle))
x1_right = int(x0_right + 200 * np.cos(angle))
y1_right = int(y0_right + 200 * np.sin(angle))



all_horizontals = np.array(all_horizontals)
best_lines = all_horizontals[all_horizontals[:, 4].argmax()]

# Define the line parameters (angle and starting point)
angle = -15 + all_horizontals[:, 4].argmax()
x0_top, y0_top = 0, int(best_lines[0]) -20
x0_bottom, y0_bottom = 0, int(best_lines[2]) -20

print("Angle: ", angle)
print(best_lines)

# Convert the angle to radians
angle = np.radians(angle)

# Calculate the end point of the lines
x1_top = int(x0_top + 500 * np.cos(angle))
y1_top = int(y0_top + 500 * np.sin(angle))
x1_bottom = int(x0_bottom + 500 * np.cos(angle))
y1_bottom = int(y0_bottom + 500 * np.sin(angle))



image2 = image.copy()

cv2.line(image2, (x0_left, y0_left), (x1_left, y1_left), 255, 1)
cv2.line(image2, (x0_left+1, y0_left), (x1_left+1, y1_left), 0, 1)
cv2.line(image2, (x0_left-1, y0_left), (x1_left-1, y1_left), 0, 1)

cv2.line(image2, (x0_right, y0_right), (x1_right, y1_right), 255, 1)
cv2.line(image2, (x0_right+1, y0_right), (x1_right+1, y1_right), 0, 1)
cv2.line(image2, (x0_right-1, y0_right), (x1_right-1, y1_right), 0, 1)


cv2.line(image2, (x0_top, y0_top), (x1_top, y1_top), 255, 1)
cv2.line(image2, (x0_top, y0_top-1), (x1_top, y1_top-1), 0, 1)
cv2.line(image2, (x0_top, y0_top+1), (x1_top, y1_top+1), 0, 1)

cv2.line(image2, (x0_bottom, y0_bottom), (x1_bottom, y1_bottom), 255, 1)
cv2.line(image2, (x0_bottom, y0_bottom-1), (x1_bottom, y1_bottom-1), 0, 1)
cv2.line(image2, (x0_bottom, y0_bottom+1), (x1_bottom, y1_bottom+1), 0, 1)


cv2.imshow('image', image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

Angle:  88
[  6.         -49.64948454 173.          52.72164948 102.37113402]
Angle:  -13
[  9.         -69.63157895  62.          40.87192118 110.50350013]
